# Explication du code de scraping SeLoger v12

Ce notebook explique en détail le fonctionnement du scraper SeLoger, bloc par bloc.

## 1. Imports et dépendances

Cette section importe toutes les bibliothèques nécessaires au fonctionnement du scraper.

In [ ]:
import time
import csv
import random
import logging
import argparse
import os
import sys
import traceback
from datetime import datetime
from typing import Optional, List, Dict, Set, Tuple
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock
from queue import Queue
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import (
    NoSuchElementException,
    StaleElementReferenceException,
    TimeoutException,
    WebDriverException,
    ElementClickInterceptedException
)

try:
    import undetected_chromedriver as uc
    UNDETECTED_AVAILABLE = True
except ImportError:
    UNDETECTED_AVAILABLE = False

### Explication des imports

| Bibliothèque | Rôle |
|-------------|------|
| **selenium** | Automatisation du navigateur web pour interagir avec les pages |
| **concurrent.futures** | Exécution parallèle avec plusieurs threads (workers) |
| **threading (Lock)** | Gestion des verrous pour la synchronisation entre threads |
| **queue** | File d'attente thread-safe pour les pages à réessayer |
| **undetected_chromedriver** | Version de Chrome qui évite la détection anti-bot |
| **re** | Expressions régulières pour extraire les données des textes |

---
## 2. Configuration globale

Cette section définit tous les paramètres de configuration du scraper.

In [ ]:
# URL de base pour la recherche
BASE_URL = "https://www.seloger.com/classified-search?distributionTypes=Buy&estateTypes=House,Apartment&locations=AD09FR43,AD09FR44,AD09FR45"

# Paramètres généraux
OUTPUT_DIR = "output"
MAX_RETRIES = 3
HEADLESS = False
PARALLEL_WORKERS = 3
MAX_WORKERS = 10
DEBUG_MODE = False
MISSING_DATA_INDICATOR = "N/A"

# Délais pour simuler un comportement humain
DELAY_BETWEEN_LISTINGS = (0.05, 0.15)  # Entre chaque annonce
PAGE_LOAD_WAIT = (2, 4)                 # Après chargement de page
SCROLL_DELAY = (0.2, 0.5)               # Entre les scrolls
LAZY_SCROLL_WAIT = (0.8, 1.5)           # Pour le lazy loading
FINAL_WAIT_AFTER_SCROLL = (1.0, 1.8)    # Après le scroll complet
RETRY_DELAY = (5, 10)                   # Avant un retry

# Seuils de qualité
MIN_LISTINGS_PER_PAGE = 15
MIN_COMPLETE_DATA_RATIO = 0.5

# Simulation de comportement humain
BREAK_EVERY_N_PAGES = (8, 15)   # Pause tous les 8-15 pages
BREAK_DURATION = (5, 15)         # Durée de la pause en secondes

### Explication de l'URL

L'URL de recherche contient plusieurs paramètres :
- `distributionTypes=Buy` : Biens à vendre (pas de location)
- `estateTypes=House,Apartment` : Maisons et appartements
- `locations=AD09FR43,AD09FR44,AD09FR45` : Codes des départements Loire-Atlantique et limitrophes

### Pourquoi des délais aléatoires ?

Les sites web détectent les bots par leur comportement trop régulier. En utilisant des intervalles aléatoires (tuples min/max), le scraper simule un utilisateur humain qui navigue naturellement.

---
## 3. Sélecteurs CSS et User Agents

In [ ]:
# User Agents pour simuler différents navigateurs
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
]

# Tailles d'écran courantes
VIEWPORT_SIZES = [
    (1920, 1080), (1366, 768), (1536, 864), (1440, 900),
    (1600, 900), (1280, 720), (1680, 1050),
]

# Sélecteurs CSS pour localiser les éléments HTML
SELECTORS = {
    'card': "div[data-testid='serp-core-classified-card-testid']",
    'url': "a[data-testid='card-mfe-covering-link-testid']",
    'price_container': "div[data-testid='cardmfe-price-testid']",
    'keyfacts': "div[data-testid='cardmfe-keyfacts-testid']",
    'address': "div[data-testid='cardmfe-description-box-address']",
    'tags': "div[data-testid='cardmfe-tag-testid']",
    'energy': "span[data-testid='card-mfe-energy-performance-class']",
}

### Pourquoi ces sélecteurs ?

SeLoger utilise des attributs `data-testid` pour leurs tests automatisés. Ces sélecteurs sont **plus stables** que les classes CSS qui peuvent changer lors des mises à jour du site.

| Sélecteur | Élément ciblé |
|-----------|---------------|
| `card` | La carte complète d'une annonce |
| `url` | Le lien vers la page détaillée |
| `price_container` | Le bloc contenant le prix |
| `keyfacts` | Surface, pièces, chambres, étage |
| `address` | Adresse, ville, code postal |
| `energy` | Classe énergétique (DPE) |

---
## 4. Variables globales thread-safe

In [ ]:
# Verrous pour la synchronisation entre threads
csv_lock = Lock()           # Protège l'écriture CSV
scraped_urls_lock = Lock()  # Protège l'ensemble des URLs
stats_lock = Lock()         # Protège les statistiques
retry_queue_lock = Lock()   # Protège la file de retry

# Ensemble des URLs déjà scrapées (pour éviter les doublons)
scraped_urls: Set[str] = set()

# File d'attente des pages échouées
retry_queue: Queue = Queue()

# Statistiques globales
global_stats = {
    'total_listings': 0,
    'complete_listings': 0,
    'failed_pages': set(),
    'successful_pages': set(),
    'pages_by_worker': {}
}

### Pourquoi des verrous (Lock) ?

Quand plusieurs threads (workers) travaillent en parallèle, ils peuvent essayer d'accéder aux mêmes ressources en même temps. Sans verrou :
- Deux workers pourraient écrire dans le CSV simultanément → **corruption des données**
- L'ensemble `scraped_urls` pourrait être modifié pendant une lecture → **erreurs**

Le verrou garantit qu'un seul thread accède à la ressource à la fois.

---
## 5. Gestion automatique des popups

C'est l'une des parties les plus importantes du scraper. SeLoger affiche plusieurs types de popups qui bloquent l'accès au contenu.

In [ ]:
def dismiss_all_popups(driver, worker_id: int) -> bool:
    """
    Ferme automatiquement TOUS les types de popups:
    - Consentement cookies (Usercentrics shadow DOM)
    - Dialogues modaux
    - Popups newsletter
    - Autres overlays
    
    Retourne True si un popup a été fermé.
    """
    dismissed_any = False
    
    # 1. POPUP COOKIES USERCENTRICS (Shadow DOM)
    try:
        cookie_script = """
            const root = document.querySelector('#usercentrics-root');
            if (root && root.shadowRoot) {
                const acceptBtn = root.shadowRoot.querySelector('[data-testid="uc-accept-all-button"]');
                if (acceptBtn && acceptBtn.offsetParent !== null) {
                    acceptBtn.click();
                    return true;
                }
            }
            return false;
        """
        if driver.execute_script(cookie_script):
            logger.info(f"Worker {worker_id}: ✓ Dismissed Usercentrics cookie popup")
            dismissed_any = True
            time.sleep(0.5)
    except Exception as e:
        logger.debug(f"Worker {worker_id}: Cookie popup check: {e}")
    
    # 2. DIALOGUES MODAUX GÉNÉRIQUES
    try:
        dialogs = driver.find_elements(By.CSS_SELECTOR, "[role='dialog']")
        for dialog in dialogs:
            if dialog.is_displayed():
                close_selectors = [
                    "button[aria-label*='close' i]",
                    "button[aria-label*='fermer' i]",
                    "button[class*='close']",
                    "[data-testid*='close']",
                ]
                for sel in close_selectors:
                    try:
                        close_btn = dialog.find_element(By.CSS_SELECTOR, sel)
                        if close_btn.is_displayed():
                            close_btn.click()
                            dismissed_any = True
                            break
                    except:
                        continue
    except Exception as e:
        pass
    
    # Suite de la fonction... (voir code complet)
    return dismissed_any

### Les différents types de popups gérés

| Type | Technique de fermeture |
|------|------------------------|
| **Cookies Usercentrics** | JavaScript dans le Shadow DOM |
| **Dialogues modaux** | Recherche de boutons "close/fermer" |
| **Overlays aria-modal** | Clic à l'extérieur du modal |
| **Popins/popups** | Recherche de boutons dans l'élément |
| **Overlays bloquants** | Clic direct sur l'overlay |
| **Fallback** | Touche Escape |

### Qu'est-ce que le Shadow DOM ?

Le Shadow DOM est une technique qui isole une partie du DOM. Selenium ne peut pas y accéder directement, d'où l'utilisation de JavaScript pour trouver et cliquer sur le bouton.

---
## 6. Parsing des annonces

Cette fonction extrait toutes les données d'une carte d'annonce.

In [ ]:
def parse_listing(card, page_num: int, worker_id: int) -> Dict[str, Optional[str]]:
    """Extrait toutes les données d'une carte d'annonce."""
    
    # Structure de données initiale
    data = {
        'page_num': page_num,
        'type': None,           # Appartement, Maison, etc.
        'price': None,          # Prix en euros
        'price_per_m2': None,   # Prix au m²
        'surface': None,        # Surface en m²
        'rooms': None,          # Nombre de pièces
        'bedrooms': None,       # Nombre de chambres
        'floor': None,          # Étage
        'address': None,        # Adresse complète
        'city': None,           # Ville
        'postal_code': None,    # Code postal
        'department': None,     # Département (2 premiers chiffres)
        'url': None,            # URL de l'annonce
        'energy_class': None,   # Classe énergétique (A-G)
        'is_new': False,        # Bien neuf ?
        'agency': None,         # Agence immobilière
        'confidence_score': 10, # Score de confiance (0-10)
    }
    
    # Extraction du texte brut pour les fallbacks
    raw_text = card.text
    
    # 1. EXTRACTION DE L'URL
    try:
        url_elem = card.find_element(By.CSS_SELECTOR, SELECTORS['url'])
        url = url_elem.get_attribute('href')
        if url:
            data['url'] = url if url.startswith('http') else f"https://www.seloger.com{url}"
    except NoSuchElementException:
        pass
    
    # 2. EXTRACTION DU PRIX (avec regex)
    try:
        price_elem = card.find_element(By.CSS_SELECTOR, SELECTORS['price_container'])
        price_text = price_elem.text
        
        # Prix au m²
        price_m2_match = re.search(r'([\d\s\u00a0\u202f,\.]+\s*€\s*/\s*m[²2])', price_text)
        if price_m2_match:
            data['price_per_m2'] = price_m2_match.group(1)
        
        # Prix principal
        main_price_match = re.search(r'([\d\s\u00a0\u202f]+)\s*€(?!\s*/)', price_text)
        if main_price_match:
            data['price'] = f"{main_price_match.group(1).strip()} €"
    except NoSuchElementException:
        pass
    
    # Suite... (voir code complet)
    return data

### Expressions régulières utilisées

| Pattern | Données extraites | Exemple |
|---------|-------------------|----------|
| `(\d+(?:[,\.]\d+)?)\s*m[²2]` | Surface | "85 m²" → "85" |
| `(\d+)\s*pièces?` | Nombre de pièces | "4 pièces" → "4" |
| `(\d+)\s*chambres?` | Nombre de chambres | "2 chambres" → "2" |
| `\((\d{5})\)` | Code postal | "(44000)" → "44000" |
| `([\d\s]+)\s*€` | Prix | "350 000 €" → "350 000" |

### Score de confiance

Le score (0-10) indique la qualité des données extraites :
- **10** : URL + Prix + Surface trouvés
- **7** : 2 champs critiques trouvés
- **4** : 1 champ critique trouvé
- **1** : Aucun champ critique
- **0** : Erreur lors du parsing

---
## 7. Configuration du navigateur Chrome

In [ ]:
def setup_chrome_driver(worker_id: int, headless: bool = False) -> webdriver:
    """Configure un navigateur Chrome pour éviter la détection."""
    
    # Sélection d'un User-Agent aléatoire
    user_agent = USER_AGENTS[worker_id % len(USER_AGENTS)]
    
    if UNDETECTED_AVAILABLE:
        # Utilise undetected-chromedriver (meilleure option)
        options = uc.ChromeOptions()
        options.add_argument(f"--user-agent={user_agent}")
        if headless:
            options.add_argument("--headless=new")
        driver = uc.Chrome(options=options)
    else:
        # Fallback sur Selenium standard avec options anti-détection
        options = ChromeOptions()
        options.add_argument(f"--user-agent={user_agent}")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        driver = webdriver.Chrome(options=options)
        
        # Supprime la propriété webdriver
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    # Taille de fenêtre aléatoire
    width, height = random.choice(VIEWPORT_SIZES)
    driver.set_window_size(width, height)
    
    # Positionnement en grille pour voir tous les workers
    x_offset = (worker_id % 3) * 650
    y_offset = (worker_id // 3) * 450
    driver.set_window_position(x_offset, y_offset)
    
    return driver

### Techniques anti-détection

| Technique | Explication |
|-----------|-------------|
| **User-Agent rotatif** | Chaque worker simule un navigateur différent |
| **disable-blink-features** | Désactive les flags d'automatisation Chrome |
| **excludeSwitches** | Supprime le switch "enable-automation" |
| **navigator.webdriver** | Supprime la propriété JS qui trahit Selenium |
| **Viewport aléatoire** | Simule différentes résolutions d'écran |
| **undetected-chromedriver** | Version patchée de Chrome plus difficile à détecter |

---
## 8. Défilement intelligent pour le lazy loading

In [ ]:
def scroll_to_load_all_cards(driver, worker_id: int, page_num: int) -> int:
    """Défile la page pour charger toutes les annonces (lazy loading)."""
    
    scroll_steps = 5  # Nombre d'étapes de scroll
    last_card_count = 0
    stable_count = 0
    
    for step in range(scroll_steps):
        # Calcul de la position cible
        scroll_position = (step + 1) * (1.0 / scroll_steps)
        target_y = int(driver.execute_script("return document.body.scrollHeight") * scroll_position)
        current_y = driver.execute_script("return window.pageYOffset")
        
        distance = target_y - current_y
        num_increments = random.randint(8, 15)
        
        # Scroll progressif avec easing cubique
        for i in range(num_increments):
            progress = (i + 1) / num_increments
            eased_progress = 1 - pow(1 - progress, 3)  # Ease-out cubic
            next_y = int(current_y + (distance * eased_progress))
            driver.execute_script(f"window.scrollTo({{top: {next_y}, behavior: 'smooth'}});")
            time.sleep(random.uniform(0.03, 0.1))
        
        time.sleep(random.uniform(*LAZY_SCROLL_WAIT))
        
        # Vérification des popups pendant le scroll
        check_and_dismiss_popups_if_needed(driver, worker_id)
        
        # Compte les cartes chargées
        cards = driver.find_elements(By.CSS_SELECTOR, SELECTORS['card'])
        current_count = len(cards)
        
        # Arrêt si le nombre de cartes est stable
        if current_count == last_card_count:
            stable_count += 1
            if stable_count >= 2:
                break
        else:
            stable_count = 0
        last_card_count = current_count
    
    # Retour en haut de page
    driver.execute_script("window.scrollTo({top: 0, behavior: 'smooth'});")
    time.sleep(random.uniform(*FINAL_WAIT_AFTER_SCROLL))
    
    return len(driver.find_elements(By.CSS_SELECTOR, SELECTORS['card']))

### Fonction d'easing cubique

La formule `1 - pow(1 - progress, 3)` crée un mouvement naturel :
- **Début** : le scroll démarre rapidement
- **Fin** : le scroll ralentit progressivement

C'est exactement comme un humain qui fait défiler une page avec la molette de souris.

```
Vitesse
  │
  │  ╭──────╮
  │ ╱        ╲
  │╱          ╲____
  └──────────────────► Temps
```

---
## 9. Gestion du fichier CSV

In [ ]:
def initialize_csv(filename: str):
    """Crée le fichier CSV avec les en-têtes."""
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    filepath = os.path.join(OUTPUT_DIR, filename)
    
    with open(filepath, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow([
            "Page_Number", "Type", "Price", "Price_Per_M2", "Surface_m2",
            "Rooms", "Bedrooms", "Floor", "Address", "City",
            "PostalCode", "Department", "Energy_Class", "Is_New",
            "Agency", "URL", "Confidence_Score"
        ])


def write_listings_to_csv(listings: List[Dict], output_file: str):
    """Écrit les annonces dans le CSV de manière thread-safe."""
    with csv_lock:  # Verrou pour éviter les conflits
        filepath = os.path.join(OUTPUT_DIR, output_file)
        with open(filepath, "a", newline="", encoding="utf-8-sig") as f:
            writer = csv.writer(f)
            for listing in listings:
                writer.writerow([
                    format_for_csv(str(listing['page_num'])),
                    format_for_csv(listing['type']),
                    format_for_csv(listing['price']),
                    # ... autres champs
                ])

### Encodage UTF-8 BOM

L'encodage `utf-8-sig` ajoute un BOM (Byte Order Mark) au début du fichier. Cela permet à **Excel** d'ouvrir correctement le fichier avec les caractères français (accents, €, etc.).

Sans le BOM, Excel pourrait afficher des caractères corrompus comme `Ã©` au lieu de `é`.

---
## 10. Fonction principale du worker

In [ ]:
def worker_scrape_pages(worker_id: int, driver: webdriver, pages: List[int], output_file: str) -> Dict:
    """Boucle principale de scraping pour un worker."""
    
    results = {
        'listings': 0, 'complete': 0,
        'failed_pages': [], 'successful_pages': [],
    }
    
    pages_since_break = 0
    next_break_at = random.randint(*BREAK_EVERY_N_PAGES)
    
    for page_idx, page_num in enumerate(pages):
        try:
            # Pause aléatoire tous les 8-15 pages
            pages_since_break += 1
            if pages_since_break >= next_break_at:
                break_time = random.uniform(*BREAK_DURATION)
                logger.info(f"Worker {worker_id}: Taking a {break_time:.1f}s break...")
                time.sleep(break_time)
                pages_since_break = 0
                next_break_at = random.randint(*BREAK_EVERY_N_PAGES)
            
            # Chargement de la page
            url = f"{BASE_URL}&page={page_num}"
            driver.get(url)
            time.sleep(random.uniform(*PAGE_LOAD_WAIT))
            
            # Fermeture des popups
            ensure_popups_dismissed(driver, worker_id)
            
            # Scroll pour charger les cartes
            card_count = scroll_to_load_all_cards(driver, worker_id, page_num)
            
            if card_count == 0:
                results['failed_pages'].append(page_num)
                retry_queue.put((page_num, worker_id, 1))
                continue
            
            # Parsing des annonces
            cards = driver.find_elements(By.CSS_SELECTOR, SELECTORS['card'])
            listings = []
            
            for card in cards:
                time.sleep(random.uniform(*DELAY_BETWEEN_LISTINGS))
                data = parse_listing(card, page_num, worker_id)
                
                # Évite les doublons
                if not is_duplicate_url(data.get('url')):
                    listings.append(data)
            
            # Écriture dans le CSV
            write_listings_to_csv(listings, output_file)
            results['listings'] += len(listings)
            results['successful_pages'].append(page_num)
            
        except Exception as e:
            logger.error(f"Worker {worker_id}: Error on page {page_num}: {e}")
            results['failed_pages'].append(page_num)
            retry_queue.put((page_num, worker_id, 1))
    
    return results

### Flux d'exécution d'un worker

```
┌─────────────────────────────────────────┐
│           Pour chaque page              │
├─────────────────────────────────────────┤
│  1. Pause aléatoire (tous les 8-15p)    │
│              ↓                          │
│  2. Chargement de la page               │
│              ↓                          │
│  3. Fermeture des popups                │
│              ↓                          │
│  4. Scroll pour lazy loading            │
│              ↓                          │
│  5. Parsing des cartes                  │
│              ↓                          │
│  6. Filtrage des doublons               │
│              ↓                          │
│  7. Écriture CSV (thread-safe)          │
│              ↓                          │
│  8. Si échec → file de retry            │
└─────────────────────────────────────────┘
```

---
## 11. Système de retry des pages échouées

In [ ]:
def retry_failed_pages(drivers: List[Tuple[int, webdriver]], output_file: str) -> Dict:
    """Réessaie les pages qui ont échoué."""
    
    results = {'retried': 0, 'succeeded': 0, 'failed': []}
    
    if retry_queue.empty():
        return results
    
    # Récupère toutes les pages à réessayer
    pages_to_retry = []
    while not retry_queue.empty():
        page_num, original_worker, attempt = retry_queue.get_nowait()
        if attempt <= MAX_RETRIES:
            pages_to_retry.append((page_num, attempt))
    
    logger.info(f"RETRY PHASE: {len(pages_to_retry)} pages to retry")
    time.sleep(random.uniform(*RETRY_DELAY))
    
    for page_num, attempt in pages_to_retry:
        # Utilise un worker différent (aléatoire)
        worker_id, driver = random.choice(drivers)
        
        try:
            url = f"{BASE_URL}&page={page_num}"
            driver.get(url)
            # ... même logique que worker_scrape_pages
            
            if success:
                results['succeeded'] += 1
            else:
                # Remet dans la queue si tentatives restantes
                if attempt < MAX_RETRIES:
                    retry_queue.put((page_num, worker_id, attempt + 1))
                else:
                    results['failed'].append(page_num)
                    
        except Exception as e:
            if attempt < MAX_RETRIES:
                retry_queue.put((page_num, worker_id, attempt + 1))
    
    return results

### Stratégie de retry

1. **Maximum 3 tentatives** par page
2. **Délai aléatoire** avant chaque round de retry
3. **Worker différent** pour chaque tentative (évite les problèmes de cache/cookies)
4. **2 rounds de retry** maximum après le scraping principal

---
## 12. Orchestration parallèle

In [ ]:
def scrape_parallel(start_page: int, end_page: int, output_file: str, num_workers: int):
    """Orchestre le scraping parallèle."""
    
    # Initialisation
    initialize_csv(output_file)
    
    # Ouverture des navigateurs
    drivers = []
    for i in range(num_workers):
        driver = setup_chrome_driver(i)
        driver.get(BASE_URL)
        drivers.append((i, driver))
        ensure_popups_dismissed(driver, i)
    
    # Distribution des pages (round-robin)
    pages = list(range(start_page, end_page + 1))
    pages_per_worker = []
    for i in range(len(drivers)):
        worker_pages = [p for j, p in enumerate(pages) if j % len(drivers) == i]
        pages_per_worker.append(worker_pages)
    
    # Exécution parallèle
    with ThreadPoolExecutor(max_workers=len(drivers)) as executor:
        futures = []
        for (worker_id, driver), worker_pages in zip(drivers, pages_per_worker):
            future = executor.submit(
                worker_scrape_pages,
                worker_id, driver, worker_pages, output_file
            )
            futures.append((future, worker_id))
        
        # Attend la fin de tous les workers
        for future, worker_id in futures:
            result = future.result()
            logger.info(f"Worker {worker_id} finished: {result['listings']} listings")
    
    # Phase de retry
    for retry_round in range(MAX_RETRY_ROUNDS):
        if retry_queue.empty():
            break
        retry_failed_pages(drivers, output_file)
    
    # Fermeture des navigateurs
    for worker_id, driver in drivers:
        driver.quit()

### Distribution round-robin des pages

Avec 3 workers et les pages 1-10 :

| Worker 0 | Worker 1 | Worker 2 |
|----------|----------|----------|
| Page 1   | Page 2   | Page 3   |
| Page 4   | Page 5   | Page 6   |
| Page 7   | Page 8   | Page 9   |
| Page 10  |          |          |

Cette distribution équilibre la charge et évite que tous les workers accèdent aux mêmes pages consécutivement.

---
## 13. Point d'entrée principal

In [ ]:
def main():
    parser = argparse.ArgumentParser(description="SeLoger Scraper v12")
    parser.add_argument("--start", type=int, help="Start page")
    parser.add_argument("--end", type=int, help="End page")
    parser.add_argument("--workers", type=int, default=PARALLEL_WORKERS)
    parser.add_argument("--output", type=str)
    parser.add_argument("--debug", action="store_true")
    
    args = parser.parse_args()
    
    # Mode interactif si pas d'arguments
    if not args.start or not args.end:
        start = int(input("Start page: ").strip())
        end = int(input("End page: ").strip())
        workers = int(input(f"Workers (default {PARALLEL_WORKERS}): ").strip() or PARALLEL_WORKERS)
    else:
        start, end, workers = args.start, args.end, args.workers
    
    output_file = args.output or f"seloger_v12_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    
    confirm = input(f"Scrape pages {start}-{end} with {workers} workers? (y/n): ")
    if confirm.lower() == 'y':
        scrape_parallel(start, end, output_file, workers)


if __name__ == "__main__":
    main()

### Modes d'utilisation

**Ligne de commande :**
```bash
python scraper.py --start 1 --end 50 --workers 3 --debug
```

**Mode interactif :**
```
$ python scraper.py
Start page: 1
End page: 50
Workers (default 3): 3
Scrape pages 1-50 with 3 workers? (y/n): y
```

---
## Résumé de l'architecture

```
┌─────────────────────────────────────────────────────────────────────┐
│                            MAIN                                     │
│  - Parse arguments / Mode interactif                                │
│  - Appelle scrape_parallel()                                        │
└───────────────────────────────┬─────────────────────────────────────┘
                                ▼
┌─────────────────────────────────────────────────────────────────────┐
│                       SCRAPE_PARALLEL                               │
│  - Initialise CSV                                                   │
│  - Ouvre N navigateurs Chrome                                       │
│  - Distribue les pages (round-robin)                                │
│  - Lance ThreadPoolExecutor                                         │
└───────────────────────────────┬─────────────────────────────────────┘
                                ▼
┌──────────────┐  ┌──────────────┐  ┌──────────────┐
│   Worker 0   │  │   Worker 1   │  │   Worker 2   │
│              │  │              │  │              │
│  Pages 1,4,7 │  │  Pages 2,5,8 │  │  Pages 3,6,9 │
│      ↓       │  │      ↓       │  │      ↓       │
│  parse_      │  │  parse_      │  │  parse_      │
│  listing()   │  │  listing()   │  │  listing()   │
└──────┬───────┘  └──────┬───────┘  └──────┬───────┘
       │                 │                 │
       └────────────┬────┴────────────────┘
                    ▼
         ┌─────────────────────┐
         │    CSV (thread-     │
         │    safe avec Lock)  │
         └─────────────────────┘
```

---
## Points forts du scraper

| Caractéristique | Avantage |
|-----------------|----------|
| **Parallélisation** | 3x plus rapide qu'un scraper séquentiel |
| **Anti-détection** | User-agents rotatifs, délais humains, undetected-chromedriver |
| **Robustesse** | Gestion des erreurs, système de retry, fallbacks multiples |
| **Gestion des popups** | Fermeture automatique de tous types de popups |
| **Thread-safety** | Verrous pour éviter les corruptions de données |
| **Score de confiance** | Permet de filtrer les données de mauvaise qualité |
| **Compatibilité Excel** | Encodage UTF-8 BOM pour les caractères français |